In [1]:
# Импорт библиотек
import numpy as np
import pandas as pd

from funcs import fill_with_mode, fill_na_with_similar, get_bounds

In [2]:
# Загрузка трёх баз данных
db1 = pd.read_excel('proc_db1.xlsx')
db2 = pd.read_excel('proc_db2.xlsx')
db3 = pd.read_excel('proc_db3.xlsx')

In [3]:
db1.head()

,Material type,Elements,Electronegativity,Ionic radius,Core size (nm),Hydro size (nm),Surface charge (mV),Surface area (m2/g),Cell type,Exposure dose (ug/mL),Number of atoms,Molecular weight (g/mol),Topological polar surface area (Å²),Density (g/cm3),Viability (%)
0,CuO,Cu,1.90,73.0,12.8,313.8,7.6,74.2,1,200.0,2,79.5450,17.1,5.94,9.1
1,ZnO,Zn,1.65,74.0,22.6,114.7,28.8,47.4,0,200.0,2,81.3800,17.1,5.44,9.5
2,ZnO,Zn,1.65,74.0,22.6,114.7,28.8,47.4,0,100.0,2,81.3800,17.1,5.44,10.2
3,ZnO,Zn,1.65,74.0,22.6,69.4,28.8,47.4,1,100.0,2,81.3800,17.1,5.44,11.0
4,Mn2O3,Mn,1.55,58.0,51.5,291.7,-46.1,25.9,1,200.0,5,157.8743,43.4,4.59,11.3


In [4]:
db2.head()

,Nanoparticle,coat,Diameter (nm),Concentration μM,Zeta potential (mV),Cells,Cell line (L)/primary cells (P),Human(H)/Animal(A) cells,Cell morphology,Cell-organ/tissue source,...,Test,Test indicator,Biochemical metric,% Cell viability,Interference checked (Y/N),Colloidal stability checked (Y/N),Positive control (Y/N),Animal,is_inorganic,Cell age
0,CeO2,nothing,20.3,0.002478,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,105.58918,N,N,N,rat,1,A
1,CeO2,nothing,20.3,0.004956,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,103.02621,N,N,N,rat,1,A
2,CeO2,nothing,20.3,0.009911,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,104.85748,N,N,N,rat,1,A
3,CeO2,nothing,20.3,0.019823,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,100.09670,N,N,N,rat,1,A
4,CeO2,nothing,20.3,0.039645,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,97.53237,N,N,N,rat,1,A


In [5]:
db3.head()

,Material,Shape,Coat/Functional Group,Synthesis_Method,Diameter (nm),Size_in_Water (nm),Size_in_Medium (nm),Zeta_in_Water (mV),Zeta_in_Medium (mV),Cell_Type,...,Cell_Morphology,Cell_Age,Cell Line_Primary Cell,Time (hr),Concentration (ug/ml),Test,Test_Indicator,Aspect_Ratio,Cell_Viability (%),is_inorganic
0,SiO2,Sphere,CTAB,Precipitation,21.06,386.9,90.09,-35.9,-28.0,PC12,...,Irregular,Adult,L,12,1.95,MTT,TetrazoliumSalt,2.17,113.670,1
1,SiO2,Sphere,CTAB,Precipitation,21.06,386.9,90.09,-35.9,-28.0,PC12,...,Irregular,Adult,L,12,3.90,MTT,TetrazoliumSalt,2.17,97.265,1
2,SiO2,Sphere,CTAB,Precipitation,21.06,386.9,90.09,-35.9,-28.0,PC12,...,Irregular,Adult,L,12,7.80,MTT,TetrazoliumSalt,2.17,10873.000,1
3,SiO2,Sphere,CTAB,Precipitation,21.06,386.9,90.09,-35.9,-28.0,PC12,...,Irregular,Adult,L,12,15.60,MTT,TetrazoliumSalt,2.17,98.451,1
4,SiO2,Sphere,CTAB,Precipitation,21.06,386.9,90.09,-35.9,-28.0,PC12,...,Irregular,Adult,L,12,31.20,MTT,TetrazoliumSalt,2.17,98.056,1


In [6]:
# Просмотри органических соединений из БД2
db2['Nanoparticle'][db2['is_inorganic'] == 0].unique()

array(['Eudragit RL', 'Dendrimer', 'PLGA', 'Polystyrene', 'Liposomes',
       'SLN', 'Carbon Nanotubes', 'Carbon NP', 'Chitosan'], dtype=object)

In [7]:
# Просмотри всех типов веществ из 1-й БД
db1['Material type'].unique()

array(['CuO', 'ZnO', 'Mn2O3', 'CoO', 'CeO2', 'Fe2O3', 'Gd2O3', 'HfO2',
       'In2O3', 'La2O3', 'NiO', 'Sb2O3', 'SiO2', 'Al2O3'], dtype=object)

In [8]:
# Добавление признака в 1-ую БД
db1['is_inorganic'] = np.ones(len(db1))

In [9]:
db3['Material'].unique()

array(['SiO2', 'QD', 'Fe2O3', 'Bi', 'Au', 'Ag', 'TiO2', 'Dendrimer',
       'PLGA', 'Alginate', 'PTFE-PMMA', 'ZnO', 'Pt', 'CuO', 'CeO2',
       'Co3O4', 'Carbon NT', 'HAP', 'MgO', 'Polystyrene', 'Ni',
       'Chitosan', 'Al2O3', 'SLN', 'EudragitRL', 'Carbon', 'ZrO2', 'C60',
       'C70', 'Diamond', 'Graphite', 'NiO', 'Cu', 'Bi2O3'], dtype=object)

In [10]:
# Добавление признака органичности в 3-ю БД
organic = [
    'EudragitRL', 'Dendrimer', 'Polystyrene',
    'Carbon', 'Chitosan', 'HAP', 'Carbon NT', 'PLGA',
    'SLN', 'Alginate'
]

db3['is_inorganic'] = db3['Material'].apply(lambda x: x not in organic)

In [11]:
# Признак источника клетки (Животное/Человек)
db1['Cell_Source'] = ['A'] * len(db1)

In [12]:
db3.rename({
    'Human_Animal': 'Cell_Source'
}, axis=1, inplace=True)

In [13]:
db2['Nanoparticle'].unique()

array(['CeO2', 'Se', 'CuO', 'CuS', 'ZnO', 'Fe2O3', 'TiO2', 'Ti', 'Cr',
       'Co', 'Co3O4', 'CdO', 'MnO', 'Bi', 'Mo', 'Pt', 'Hydroxyapatite',
       'Cu2O', 'MgO', 'Al2O3', 'QDs', 'SiO2', 'Au', 'Ag', 'Eudragit RL',
       'Dendrimer', 'PLGA', 'Polystyrene', 'Liposomes', 'SLN',
       'Carbon Nanotubes', 'Carbon NP', 'Chitosan'], dtype=object)

In [14]:
from pymatgen.core.composition import Composition
from funcs import get_electroneg

# Новые признаки для БД 1-3:
# 1) Электроотрицательность
# 2) Молекулярная масса

other = {
    'Diamond': 12,
    'Graphite': 12,
    'Polystyrene': 104,
    'Carbon': 12,
    'Carbon NT': 12,
    'EudraginRL': 231
}

for mat, db in zip(['Material', 'Nanoparticle'], [db3, db2]):
    weights = []

    for formula in db[mat]:
        try:
            comp = Composition(formula)
            weights.append(comp.weight)
        except:
            if formula in other:
                weights.append(other[formula])
            else:
                weights.append(np.nan)

    db['Molecular weight (g/mol)'] = np.array(weights)
    db['Electronegativity'] = get_electroneg(db[mat])
    fill_with_mode(db['Molecular weight (g/mol)'])
    fill_with_mode(db['Electronegativity'])

d:\python\lib\site-packages\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ds. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


In [15]:
# Изменение единиц измерения концентрации во 2-й БД
db2['Concentration (ug/mL)'] = db2['Concentration μM'] * db2['Molecular weight (g/mol)'] * 1000

In [16]:
# Переименование
db1.rename({
    'Surface charge (mV)': 'Zeta_in_Water (mV)',
    'Material type': 'Nanoparticle',
    'Exposure dose (ug/mL)': 'Concentration (ug/mL)',
    'Hydro size (nm)': 'Diameter (nm)'
}, inplace=True, axis=1)

In [17]:
db2.rename({
    '% Cell viability': 'Viability (%)',
    'Human(H)/Animal(A) cells': 'Cell_Source',
    'Exposure time (h)': 'Time (hr)',
    'Cell-organ/tissue source': 'Cell_Tissue',
    'Cell morphology': 'Cell_Morphology',
    'Test indicator': 'Test_Indicator',
    'Cell age': 'Cell_Age',
    'Zeta potential (mV)': 'Zeta_in_Water (mV)',
    'Cells': 'Cell_Type'
}, inplace=True, axis=1)

In [18]:
db3.rename({
    'Material': 'Nanoparticle',
    'Cell_Viability (%)': 'Viability (%)',
    'Concentration (ug/ml)': 'Concentration (ug/mL)',
    'Coat/Functional Group': 'coat',
    'Cell Line_Primary Cell': 'Cell line (L)/primary cells (P)'
}, inplace=True, axis=1)

In [19]:
# Все колонки в 1-3 БД
np.array(list(set(db1.columns) | set(db2.columns) | set(db3.columns)))

array(['Cell_Source', 'Topological polar surface area (Å²)',
       'Cell line (L)/primary cells (P)', 'Core size (nm)',
       'Zeta_in_Medium (mV)', 'Synthesis_Method', 'Number of atoms',
       'Size_in_Medium (nm)', 'Animal', 'Nanoparticle', 'Cell_Type',
       'is_inorganic', 'Cell_Morphology', 'Test_Indicator', 'Time (hr)',
       'Zeta_in_Water (mV)', 'Biochemical metric', 'Density (g/cm3)',
       'Electronegativity', 'Cell_Tissue', 'Cell_Age', 'Concentration μM',
       'Concentration (ug/mL)', 'Aspect_Ratio', 'Shape',
       'Molecular weight (g/mol)', 'Interference checked (Y/N)',
       'Colloidal stability checked (Y/N)', 'Diameter (nm)',
       'Viability (%)', 'Size_in_Water (nm)', 'coat',
       'Positive control (Y/N)', 'Ionic radius', 'Test',
       'Surface area (m2/g)', 'Elements', 'Cell type'], dtype='<U35')

In [20]:
# Ненужные колонки
garbage = set([
    'Number of atoms', 'Colloidal stability checked (Y/N)',
    'Biochemical metric', 'Cell type', 'Elements', 'Interference checked (Y/N)',
    'Cell Line_Primary Cell', 'Aspect_Ratio', 'Ionic radius', 'Concentration μM',
    'Positive control (Y/N)', 'Core size (nm)', 'Animal', 'Synthesis_Method',
    'Shape', 'Topological polar surface area (Å²)'
])

In [21]:
# Удаление ненужных колонок из БД 1-3
all_cols = set()

for db in [db1, db2, db3]:
    cols = list(set(db.columns) & garbage)
    db.drop(cols, axis=1, inplace=True)
    all_cols |= set(db.columns)

In [22]:
all_cols

{'Cell line (L)/primary cells (P)',
 'Cell_Age',
 'Cell_Morphology',
 'Cell_Source',
 'Cell_Tissue',
 'Cell_Type',
 'Concentration (ug/mL)',
 'Density (g/cm3)',
 'Diameter (nm)',
 'Electronegativity',
 'Molecular weight (g/mol)',
 'Nanoparticle',
 'Size_in_Medium (nm)',
 'Size_in_Water (nm)',
 'Surface area (m2/g)',
 'Test',
 'Test_Indicator',
 'Time (hr)',
 'Viability (%)',
 'Zeta_in_Medium (mV)',
 'Zeta_in_Water (mV)',
 'coat',
 'is_inorganic'}

In [23]:
# Добавление недостающих колонок для БД 1-3
for db in [db1, db2, db3]:
    to_add = all_cols - set(db.columns)
    for ta in to_add:
        db[ta] = [np.nan] * len(db)

In [24]:
# Соединение
db123 = pd.concat([db1, db2, db3], axis=0)
db123.reset_index(inplace=True, drop=True)

In [25]:
db123.head()

,Nanoparticle,Electronegativity,Diameter (nm),Zeta_in_Water (mV),Surface area (m2/g),Concentration (ug/mL),Molecular weight (g/mol),Density (g/cm3),Viability (%),is_inorganic,...,Cell_Type,Cell_Morphology,Test_Indicator,Time (hr),Test,Cell line (L)/primary cells (P),Zeta_in_Medium (mV),Cell_Tissue,Size_in_Medium (nm),Size_in_Water (nm)
0,CuO,1.90,313.8,7.6,74.2,200.0,79.5450,5.94,9.1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZnO,1.65,114.7,28.8,47.4,200.0,81.3800,5.44,9.5,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZnO,1.65,114.7,28.8,47.4,100.0,81.3800,5.44,10.2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZnO,1.65,69.4,28.8,47.4,100.0,81.3800,5.44,11.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mn2O3,1.55,291.7,-46.1,25.9,200.0,157.8743,4.59,11.3,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Загрузка соединённых БД 4-5
db45 = pd.read_excel('Database_45.xlsx')
db45.head()

,Material type,Viability (%),Ionic radius,Hydro size (nm),Core size (nm),Concentration (ug/mL),Zeta potential (mV),Surface area (m2/g),Exposure time (h),Molecular weight (g/mol),Electronegativity,Density (g/cm3),Cell-organ/tissue source
0,CuO,9.1,73,313.8,12.8,200.0,7.6,74.2,NaN,79.545,1.9,5.94,NaN
1,ZnO,9.5,74,114.7,22.6,200.0,28.8,47.4,NaN,81.38,1.65,5.44,NaN
2,ZnO,10.2,74,114.7,22.6,100.0,28.8,47.4,NaN,81.38,1.65,5.44,NaN
3,ZnO,11.0,74,69.4,22.6,100.0,28.8,47.4,NaN,81.38,1.65,5.44,NaN
4,Mn2O3,11.3,58,291.7,51.5,200.0,-46.1,25.9,NaN,157.8743,1.55,4.59,NaN


In [27]:
# Переименование столбцов
db45.rename({
    'Hydro size (nm)': 'Diameter (nm)',
    'Material type': 'Nanoparticle',
    'Exposure time (h)' : 'Time (hr)'
}, axis=1, inplace=True)

In [28]:
# Добавление столбцов
db45['Cell_Source'] = ['A'] * len(db45)
db45['Density (g/cm3)'] = [np.nan] * len(db45)

In [29]:
# Добавление столбца органичности для БД 4-5
organic = db123['Nanoparticle'][db123['is_inorganic'] == 0].values
organic

array(['Eudragit RL', 'Eudragit RL', 'Eudragit RL', ..., 'Polystyrene',
       'Polystyrene', 'Polystyrene'], dtype=object)

In [30]:
db45['is_inorganic'] = db45['Nanoparticle'].apply(lambda x: x not in organic).astype('int32')

In [31]:
db123.columns

Index(['Nanoparticle', 'Electronegativity', 'Diameter (nm)',
       'Zeta_in_Water (mV)', 'Surface area (m2/g)', 'Concentration (ug/mL)',
       'Molecular weight (g/mol)', 'Density (g/cm3)', 'Viability (%)',
       'is_inorganic', 'Cell_Source', 'Cell_Age', 'coat', 'Cell_Type',
       'Cell_Morphology', 'Test_Indicator', 'Time (hr)', 'Test',
       'Cell line (L)/primary cells (P)', 'Zeta_in_Medium (mV)', 'Cell_Tissue',
       'Size_in_Medium (nm)', 'Size_in_Water (nm)'],
      dtype='object')

In [32]:
db45.columns

Index(['Nanoparticle', 'Viability (%)', 'Ionic radius', 'Diameter (nm)',
       'Core size (nm)', 'Concentration (ug/mL)', 'Zeta potential (mV)',
       'Surface area (m2/g)', 'Time (hr)', 'Molecular weight (g/mol)',
       'Electronegativity', 'Density (g/cm3)', 'Cell-organ/tissue source',
       'Cell_Source', 'is_inorganic'],
      dtype='object')

In [33]:
# Ещё переименование
db45.rename({
    'Cell-organ/tissue source': 'Cell_Tissue',
    'Zeta potential (mV)': 'Zeta_in_Water (mV)'
}, axis=1, inplace=True)

In [34]:
# Удаление столбцов из БД 4-5, которых нет в БД 1-3
to_remove = [col for col in db45.columns if col not in db123.columns]
to_remove

['Ionic radius', 'Core size (nm)']

In [35]:
db45.drop(to_remove, axis=1, inplace=True)

In [36]:
# Добавление в БД 4-5 недостающих колонок
for col in set(db123.columns) - set(db45.columns):
    db45[col] = [np.nan] * len(db45)

In [37]:
# Соединение
db = pd.concat([db123, db45], axis=0)
db.reset_index(drop=True, inplace=True)
db.head()

,Nanoparticle,Electronegativity,Diameter (nm),Zeta_in_Water (mV),Surface area (m2/g),Concentration (ug/mL),Molecular weight (g/mol),Density (g/cm3),Viability (%),is_inorganic,...,Cell_Type,Cell_Morphology,Test_Indicator,Time (hr),Test,Cell line (L)/primary cells (P),Zeta_in_Medium (mV),Cell_Tissue,Size_in_Medium (nm),Size_in_Water (nm)
0,CuO,1.9,313.8,7.6,74.2,200.0,79.545,5.94,9.1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZnO,1.65,114.7,28.8,47.4,200.0,81.38,5.44,9.5,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZnO,1.65,114.7,28.8,47.4,100.0,81.38,5.44,10.2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZnO,1.65,69.4,28.8,47.4,100.0,81.38,5.44,11.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mn2O3,1.55,291.7,-46.1,25.9,200.0,157.8743,4.59,11.3,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
db.isna().sum()

Nanoparticle                          0
Electronegativity                    18
Diameter (nm)                         0
Zeta_in_Water (mV)                    0
Surface area (m2/g)                7007
Concentration (ug/mL)                 0
Molecular weight (g/mol)             18
Density (g/cm3)                    8649
Viability (%)                         0
is_inorganic                          0
Cell_Source                           0
Cell_Age                           2136
coat                               2136
Cell_Type                          2136
Cell_Morphology                    2136
Test_Indicator                     2136
Time (hr)                          1562
Test                               2136
Cell line (L)/primary cells (P)    2136
Zeta_in_Medium (mV)                5032
Cell_Tissue                        1562
Size_in_Medium (nm)                5032
Size_in_Water (nm)                 5032
dtype: int64

In [39]:
# Заполнение столбца плотности
fill_na_with_similar(db, 'Density (g/cm3)', 'Nanoparticle')

0.3938027517632096

In [40]:
import json

with open('densities.json', 'r') as f:
    densities = json.load(f)

len(densities)

31

In [41]:
dens_vals = []
ids = np.where(db['Density (g/cm3)'].isna())[0]

for i in ids:
    formula = db['Nanoparticle'][i]
    if formula in densities:
        dens_vals.append(densities[formula])
    else:
        dens_vals.append(np.nan)

db.loc[ids, 'Density (g/cm3)'] = dens_vals
db['Density (g/cm3)'].isna().sum() / len(db)

0.13245105545225855

In [42]:
fill_with_mode(db['Density (g/cm3)'])

In [43]:
db.columns

Index(['Nanoparticle', 'Electronegativity', 'Diameter (nm)',
       'Zeta_in_Water (mV)', 'Surface area (m2/g)', 'Concentration (ug/mL)',
       'Molecular weight (g/mol)', 'Density (g/cm3)', 'Viability (%)',
       'is_inorganic', 'Cell_Source', 'Cell_Age', 'coat', 'Cell_Type',
       'Cell_Morphology', 'Test_Indicator', 'Time (hr)', 'Test',
       'Cell line (L)/primary cells (P)', 'Zeta_in_Medium (mV)', 'Cell_Tissue',
       'Size_in_Medium (nm)', 'Size_in_Water (nm)'],
      dtype='object')

In [44]:
# Удаление ненужных колонок и заполнение NaN значений
db.drop('Surface area (m2/g)', axis=1, inplace=True)
fill_with_mode(db['Electronegativity'])
fill_with_mode(db['Molecular weight (g/mol)'])
fill_with_mode(db['Time (hr)'])

In [45]:
cols = ['Size_in_Water (nm)', 'Size_in_Medium (nm)',
    'Cell line (L)/primary cells (P)', 'Zeta_in_Medium (mV)']

for col in cols:
    score = fill_na_with_similar(db, col, 'Nanoparticle')
    print(col, score)
    fill_with_mode(db[col])

Size_in_Water (nm) 0.7792130365659777
Size_in_Medium (nm) 0.7792130365659777
Cell line (L)/primary cells (P) 0.7443820224719101
Zeta_in_Medium (mV) 0.7792130365659777


In [46]:
db.isna().sum()

Nanoparticle                          0
Electronegativity                     0
Diameter (nm)                         0
Zeta_in_Water (mV)                    0
Concentration (ug/mL)                 0
Molecular weight (g/mol)              0
Density (g/cm3)                       0
Viability (%)                         0
is_inorganic                          0
Cell_Source                           0
Cell_Age                           2136
coat                               2136
Cell_Type                          2136
Cell_Morphology                    2136
Test_Indicator                     2136
Time (hr)                             0
Test                               2136
Cell line (L)/primary cells (P)       0
Zeta_in_Medium (mV)                   0
Cell_Tissue                        1562
Size_in_Medium (nm)                   0
Size_in_Water (nm)                    0
dtype: int64

In [47]:
# Просмотр всех колонок с категориальным типом данных
[col for col in db.columns if db[col].dtype == 'object']

['Nanoparticle',
 'Electronegativity',
 'Molecular weight (g/mol)',
 'Cell_Source',
 'Cell_Age',
 'coat',
 'Cell_Type',
 'Cell_Morphology',
 'Test_Indicator',
 'Test',
 'Cell line (L)/primary cells (P)',
 'Cell_Tissue']

In [48]:
db['Nanoparticle'].unique()

array(['CuO', 'ZnO', 'Mn2O3', 'CoO', 'CeO2', 'Fe2O3', 'Gd2O3', 'HfO2',
       'In2O3', 'La2O3', 'NiO', 'Sb2O3', 'SiO2', 'Al2O3', 'Se', 'CuS',
       'TiO2', 'Ti', 'Cr', 'Co', 'Co3O4', 'CdO', 'MnO', 'Bi', 'Mo', 'Pt',
       'Hydroxyapatite', 'Cu2O', 'MgO', 'QDs', 'Au', 'Ag', 'Eudragit RL',
       'Dendrimer', 'PLGA', 'Polystyrene', 'Liposomes', 'SLN',
       'Carbon Nanotubes', 'Carbon NP', 'Chitosan', 'QD', 'Alginate',
       'PTFE-PMMA', 'Carbon NT', 'HAP', 'Ni', 'EudragitRL', 'Carbon',
       'ZrO2', 'C60', 'C70', 'Diamond', 'Graphite', 'Cu', 'Bi2O3',
       'Fe3O4'], dtype=object)

In [49]:
# Исправление того, что разные названия
# обозначают одно и то же
db['Nanoparticle'].replace({
    'Carbon Nanotubes': 'Carbon NT'
}, inplace=True)

In [50]:
# Удаление "глупостей"
for col in ['Electronegativity', 'Molecular weight (g/mol)']:
    for i, el in enumerate(db[col]):
        if type(el) == str:
            db.loc[i, col] = float(el.replace(',', '.'))

In [51]:
db['Cell_Source'].unique()

array(['A', 'H'], dtype=object)

In [52]:
db['Test'].unique()

array([nan, 'MTT', 'XTT', 'NR', 'LDH', 'MTS', 'Live/Dead', 'WST\xad1',
       'CellTiter\xadBlue', 'CellTiter\xadGlo', 'CellTiter\xadBlue ',
       'ATPLite', 'Alamar Blue', 'Alamar blue', 'ATP', 'WST\xad8',
       'Resazurin', 'ApoTox\xadGlo™ Triplex', 'CytoTox\xadOne™',
       'Vialight', 'Promegas CTB Assay',
       'Modified MTT assay (MTT\xadformazan ppt dissolving by ethanol)',
       'Modified MTT assay (MTT\xadformazan ppt dissolving by isopropanol/HCl)',
       'Live/Dead ', 'CellTiterBlue', 'NRU', 'AlamarBlue', 'CCK-8', 'CVS',
       'WST-1', 'CellTiterGlo', 'CytoTox-Glo', 'EZ4U', 'CoulterCounter'],
      dtype=object)

In [53]:
db['Test'].replace({
    'Live/Dead ': 'Live/Dead',
    'Alamar blue': 'Alamar Blue',
    'CellTiter\\xadBlue ': 'CellTiter\\xadBlue'
}, inplace=True)

In [54]:
db['Test_Indicator'].unique()

array([nan, 'tetrazolium salt', 'toluylene red', 'LDH activity assay kit',
       'Calcein AM, Ethidium homodimer-1', 'Sytox Red', 'Annexin V',
       'propidium iodide', 'trypan blue', 'Alamar blue', 'luciferase',
       'propidium iodide, Annexin V', 'Caspase-Glo® 3/7 Reagent',
       'Trypan Blue', 'Propidium iodide', 'calcein AM',
       'Hoechst 33358, propidium iodide', 'Calcein AM, propidium iodide',
       'TetrazoliumSalt', 'AlamarBlue', 'LDHrelease', 'PropidiumIodide',
       'CalceinAM', 'NeutralRed', 'TrypanBlue', 'CrystalViolet',
       'LuciferaseEnzyme', 'Dead-Cell-Protease', 'Impedance'],
      dtype=object)

In [55]:
db['Test_Indicator'].replace({
    'Trypan Blue': 'trypan blue',
    'TrypanBlue': 'trypan blue',
    'TetrazoliumSalt': 'tetrazolium salt',
    'Propidium iodide': 'propidium iodide',
    'PropidiumIodide': 'propidium iodide',
    'CalceinAM': 'calcein AM',
    'AlamarBlue': 'Alamar blue'
}, inplace=True)

In [56]:
db['Cell_Tissue'] = db['Cell_Tissue'].apply(lambda x: x if type(x) != str else x.lower())

In [57]:
db['Cell_Tissue'].unique()

array([nan, 'areolar tissue', 'prostate', 'lung', 'heart', 'bone marrow',
       'brain', 'kidney', 'skin', 'blood', 'colon', 'ovary', 'cervix',
       'embryo', 'liver', 'pituitary gland', 'breast', 'aorta', 'bone',
       'foreskin', 'adrenal gland', 'umbilical cord', 'stomach', 'testis',
       'axillary lymph node', 'urinary bladder', 'ascites',
       'umbilical vein', 'cerebellum', 'cortex', 'bonemarrow', 'cornea',
       'peritoneal cavity', 'subcutaneousconnectivetissue',
       'respiratorytract', 'retina', 'urinarybladder', 'fatpad', 'amnion',
       'teeth', 'gingiva', 'connective tissue', 'mesothelium', 'nose'],
      dtype=object)

In [58]:
db['Cell_Tissue'].replace({
    'bonemarrow': 'bone marrow'
}, inplace=True)

In [59]:
db['Cell_Age'].unique()

array([nan, 'A', 'E', 'Adult', 'Embryonic', 'Fetus'], dtype=object)

In [60]:
db['Cell_Age'].replace({
    'Adult': 'A',
    'Embryonic': 'E',
    'Fetus': 'E'
}, inplace=True)

In [61]:
db['coat'].unique()

array([nan, 'nothing', 'l-cysteine l-lysine l-lysine', 'Dextran', 'PEI',
       'PEG to the PEI', 'PEG', 'PEG-NH2', 'BSA', 'alginate', 'COOH',
       'NH2', 'dimercaptosuccinic',
       'folic acid with intermediate organic (PEG) coating',
       'folic acid with intermediate inorganic (silica) coating',
       'Digestive enzymes', 'simethicone then esters on top',
       'alumina-simethicone', 'silica', 'PVP', 'Folic acid',
       'D-penicillamine (NH2/COOH)', '3-mercaptopropionic acid (COOH)',
       'cysteamine (NH2)', 'cysteamine', 'Zn then cysteamine', 'SGF',
       'TGA-gelatine', 'TGA', 'CTAB', 'Phosphonate', 'PEG-OCH3',
       'PEG-COOH', 'Citrate and PVP', 'Citrate', 'Starch', 'Gum Arabic',
       'Star anise', 'Sodium borohydride', 'Chitosan', 'PVA',
       'Poloxamer 188 (Pluronic F68)', 'metal catalyst residues',
       'Tween 80', 'COONa', 'SO3Na', 'Hyaluronic acid ',
       'Hyaluronic acid', 'Ph', 'None', 'DPA', 'MPA', 'CyA', 'Silica',
       'CKK', 'Ac', 'C12', 'OH', 'O

In [62]:
db['coat'].replace({
    'None': 'nothing'
}, inplace=True)

In [63]:
db['Cell line (L)/primary cells (P)'].unique()

array(['L', 'P'], dtype=object)

In [64]:
db['Cell_Morphology'].unique()

array([nan, 'Fibroblast', 'Epithelial', 'Endothelial',
       'Monocyte/Macrophage', 'Lymphoblast', 'Monocyte',
       'Endothelial-like', 'Irregular', 'Spindle', 'Neuronal',
       'Macrophage', 'Mesenchymal', 'Keratinocyte', 'Polygonal',
       'Epithelial-like', 'Osteoblast', 'Mixed', 'MulticellularClusters'],
      dtype=object)

In [65]:
db['Cell_Type'].unique()

array([nan, 'L929', 'PC3', 'A549', 'HCMEC', 'SH\xadSY5Y', 'H4',
       'HEK\xad293', 'A431', 'RAW 264.7', 'Caco\xad2', 'Fibroblasts',
       'CD3+ T cells', 'CD4+T cells', 'B cells', 'NK cells', 'Monocytes',
       'T cells (all types)', 'Naive T\xadcell', 'Memory T\xadcell',
       'Lymphocytes', 'SKOV\xad3', 'Macrophages', 'HMEC\xad1', 'HeLa',
       'MEF', 'HepG2', 'GH3', 'MCF7', 'CDBgeo', 'TD', 'V14', 'PAECs',
       'T98G', 'MG\xad63', 'hTERT\xadBJ1', 'PC12', 'VERO', 'MDCK',
       'CHO\xadK1', 'IP15', 'LLC\xadPK1', 'HAEC', 'HUVEC', 'AGS',
       'BEAS\xad2B', 'Neuro\xad2a', 'C18–4', 'IMR90', 'HMM', 'L\xad02',
       'PMA activated THP\xad1', 'SVEC4\xad10', 'SK\xadBR\xad3',
       'primary tissue Macrophage', 'primary alveolar Macrophage',
       'primary alveolar Epithelial cells',
       'primary alveolar epithelial cells', 'NCIH441', 'CHO22',
       'MDA\xadMB\xad231', 'C17.2', 'hMSCs', 'HaCaT\xa0', 'LoVo', 'KEC',
       'ECV304', 'THP\xad1', 'NR8383', 'SW480', 'J774', 'Clone\x

In [66]:
db['Cell_Type'].replace({
    'primary alveolar epithelial cells': 'primary alveolar Epithelial cells',
}, inplace=True)

In [67]:
db.isna().sum()

Nanoparticle                          0
Electronegativity                     0
Diameter (nm)                         0
Zeta_in_Water (mV)                    0
Concentration (ug/mL)                 0
Molecular weight (g/mol)              0
Density (g/cm3)                       0
Viability (%)                         0
is_inorganic                          0
Cell_Source                           0
Cell_Age                           2136
coat                               2136
Cell_Type                          2136
Cell_Morphology                    2136
Test_Indicator                     2136
Time (hr)                             0
Test                               2136
Cell line (L)/primary cells (P)       0
Zeta_in_Medium (mV)                   0
Cell_Tissue                        1562
Size_in_Medium (nm)                   0
Size_in_Water (nm)                    0
dtype: int64

In [68]:
# Заполнение недостающих данных
cols = ['Test', 'Test_Indicator', 'Cell_Tissue',
    'Cell_Type', 'Cell_Morphology', 'Cell_Age', 'coat']

for col in cols:
    score = fill_na_with_similar(db, col, 'Nanoparticle')
    print(col, score)
    fill_with_mode(db[col])

Test 0.7443820224719101
Test_Indicator 0.7443820224719101
Cell_Tissue 0.6837387964148528
Cell_Type 0.7443820224719101
Cell_Morphology 0.7443820224719101
Cell_Age 0.7443820224719101
coat 0.7443820224719101


In [69]:
# Проверка
db.isna().any().any()

False

In [70]:
# Сохранение таблицы
db.to_excel('connected.xlsx', index=False)